<a href="https://colab.research.google.com/github/aryalkoshish/big_data/blob/main/Spark_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark #initializing python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=86f1ce1bf59a782eae6a4a47c205ee15f99de0c21f0c93d25bef66308d9cb401
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from google.colab import files #importing/uploading files
uploaded = files.upload()

Saving auto-mpg.csv to auto-mpg.csv


In [ ]:
from pyspark.sql import SparkSession #starting spark session

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
df = spark.read.csv('auto-mpg.csv',header=True) # reading file and storking in dataframe
df.head(5)

[Row(mpg='18', cylinders='8', displacement='307', horsepower='130', weight='3504', acceleration='12', model-year='70'),
 Row(mpg='15', cylinders='8', displacement='350', horsepower='165', weight='3693', acceleration='11.5', model-year='70'),
 Row(mpg='18', cylinders='8', displacement='318', horsepower='150', weight='3436', acceleration='11', model-year='70'),
 Row(mpg='16', cylinders='8', displacement='304', horsepower='150', weight='3433', acceleration='12', model-year='70'),
 Row(mpg='17', cylinders='8', displacement='302', horsepower='140', weight='3449', acceleration='10.5', model-year='70')]

In [ ]:
import pyspark.sql.functions as F
median= df.agg(F.median("horsepower").alias("horsepower_median"))

In [ ]:
import pyspark.sql.functions as F
df_median = df.groupby('model-year').agg(F.median("horsepower"))

In [ ]:
df = df.join(median,how ="cross")

In [ ]:
from pyspark.sql.functions import col, when
df=df.withColumn('horsepower_marker',when(col('horsepower') < col('horsepower_median'),'lower_than_median').otherwise('higher_than_median'))

In [ ]:
df.show()

+---+---------+------------+----------+------+------------+----------+-----------------+------------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model-year|horsepower_median| horsepower_marker|
+---+---------+------------+----------+------+------------+----------+-----------------+------------------+
| 18|        8|         307|       130|  3504|          12|        70|             92.0|higher_than_median|
| 15|        8|         350|       165|  3693|        11.5|        70|             92.0|higher_than_median|
| 18|        8|         318|       150|  3436|          11|        70|             92.0|higher_than_median|
| 16|        8|         304|       150|  3433|          12|        70|             92.0|higher_than_median|
| 17|        8|         302|       140|  3449|        10.5|        70|             92.0|higher_than_median|
| 15|        8|         429|       198|  4341|          10|        70|             92.0|higher_than_median|
| 14|        8|         454|